In [1]:
# Manually add the root path before running the test
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))
from utils.helpers import (
    comparator_less,
    prepare_ancilla_cell_validity,
    Indexer
)

from oracle.row_uniqueness import row_uniqueness_circuit

In [10]:
from math import ceil, log2
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator


# --- 1) PARAMETERS & GRID ---
n = 3
# every row has duplicates -> row_constraints_flag should return 1
grid = [
    0, 1, 2, 
    0, 1, 2, 
    0, 1, 2,  
]

# --- 2) SET UP INDEXER + QUBIT POOL ---
# We need:
#  • data: n*n*k
#  • row_flags: n
#  • row_constraints_flag: 1
#  • comparator ancillas: k+1
k = max(1, int(ceil(log2(n))))
num_anc = (k+1) + n + 1    # k+1 for comparator, n for row flags, 1 for global flag
idx = Indexer(n, num_anc)

# One flat register for everything
qr = QuantumRegister(idx.total_qubits, name="q")
#cr = ClassicalRegister(1, name="c")   # measure only the final constraint flag
qc = QuantumCircuit(qr, name="row_uniqueness_demo")

# --- 3) LOAD GRID VALUES INTO DATA QUBITS ---
for i in range(n):
    for j in range(n):
        val = grid[i*n + j]
        for bit in range(idx.k):
            if (val >> bit) & 1:
                qc.x(qr[idx.data(i, j, bit)])

# --- 4) APPEND THE ROW‐UNIQUENESS ORACLE ---
# Flips row_flags[i] if row i has duplicates, then collapses into idx.row_flag()
row_uniqueness_circuit(qc, qr, idx)

# --- 5) MEASURE JUST THE GLOBAL ROW‐CONSTRAINT FLAG ---
#qc.measure(qr[idx.row_flag()], cr[0])
qc.measure_all()

# --- 6) SIMULATE & PRINT RESULTS ---
sim = AerSimulator(method="matrix_product_state")
tcirc = transpile(qc, sim, optimization_level=3)
result = sim.run(tcirc, shots=1).result()
counts = result.get_counts()

print("Total qubits:", qc.num_qubits)
print("Counts for row_constraints_flag:", counts)


Total qubits: 29
Counts for row_constraints_flag: {'00000000000100100100100100100': 1}


In [11]:
# after you’ve run and obtained `result.get_counts()`:
bitstring = next(iter(counts))              # e.g. '01000101…'
# Qiskit’s bitstrings are returned MSB→LSB (left→right), and qubit 0 is the rightmost bit
bits = list(map(int, bitstring[::-1]))      # now bits[i] is the measured value of qubit i

# 1) print out each qubit’s role and its measured value
print("Measured qubits:")
for q in range(idx.total_qubits):
    role = idx.pretty(q)
    print(f" q[{q:2d}] = {bits[q]}   ⟶ {role}")

# 2) reconstruct the grid from the data-region
recovered = [None] * (n*n)
for i in range(n):
    for j in range(n):
        v = 0
        for b in range(idx.k):
            bitval = bits[idx.data(i, j, b)]
            v |= (bitval << b)
        recovered[i*n + j] = v

# 3) print the grid as rows
print("\nReconstructed grid values:")
for i in range(n):
    row = recovered[i*n:(i+1)*n]
    print(" ", row)

# 4) check the final flag
final_bit = bits[idx.row_flag()]
print(f"\ncell_valid_flag  (q[{idx.row_flag()}]) = {final_bit}")


Measured qubits:
 q[ 0] = 0   ⟶ data(0,0,0)
 q[ 1] = 0   ⟶ data(0,0,1)
 q[ 2] = 1   ⟶ data(0,1,0)
 q[ 3] = 0   ⟶ data(0,1,1)
 q[ 4] = 0   ⟶ data(0,2,0)
 q[ 5] = 1   ⟶ data(0,2,1)
 q[ 6] = 0   ⟶ data(1,0,0)
 q[ 7] = 0   ⟶ data(1,0,1)
 q[ 8] = 1   ⟶ data(1,1,0)
 q[ 9] = 0   ⟶ data(1,1,1)
 q[10] = 0   ⟶ data(1,2,0)
 q[11] = 1   ⟶ data(1,2,1)
 q[12] = 0   ⟶ data(2,0,0)
 q[13] = 0   ⟶ data(2,0,1)
 q[14] = 1   ⟶ data(2,1,0)
 q[15] = 0   ⟶ data(2,1,1)
 q[16] = 0   ⟶ data(2,2,0)
 q[17] = 1   ⟶ data(2,2,1)
 q[18] = 0   ⟶ row_flag
 q[19] = 0   ⟶ col_flag
 q[20] = 0   ⟶ cell_valid_flag
 q[21] = 0   ⟶ global_flag
 q[22] = 0   ⟶ ancilla(0)
 q[23] = 0   ⟶ ancilla(1)
 q[24] = 0   ⟶ ancilla(2)
 q[25] = 0   ⟶ ancilla(3)
 q[26] = 0   ⟶ ancilla(4)
 q[27] = 0   ⟶ ancilla(5)
 q[28] = 0   ⟶ ancilla(6)

Reconstructed grid values:
  [0, 1, 2]
  [0, 1, 2]
  [0, 1, 2]

cell_valid_flag  (q[18]) = 0
